In [ ]:
import matplotlib.pyplot as plt
from keras.models import load_model, Model
import numpy as np
from utils.visualizations import GradCAM, GuidedGradCAM, GBP
from utils.visualizations import LRP, CLRP, LRPA, LRPB, LRPE
from utils.visualizations import SGLRP, SGLRPSeqA, SGLRPSeqB
from utils.helper import heatmap
import innvestigate.utils as iutils
import os
from keras.preprocessing.image import img_to_array, load_img

# limits tensorflow to a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Using TensorFlow backend.


In [ ]:
# This will be your trained model instead.

from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
model = VGG16(
    weights='imagenet'
)
# model.summary()

In [ ]:
# Only the partial model is needed for the visualizers. Use innvestigate.utils.keras.graph.pre_softmax_tensors()
partial_model = Model(
    inputs=model.inputs,
    outputs=iutils.keras.graph.pre_softmax_tensors(model.outputs),
    name=model.name,
)
partial_model.summary()

In [ ]:
# Range of input images
# These values are set due to Keras Applications. Change this to a range suitable for your model.
max_input = 151.061
min_input = -123.68

In [ ]:
# Change this to load a list of images you want. 
orig_imgs = [img_to_array(load_img("data/images/zebra_elephant{}.jpg".format(i+1), target_size=(224, 224))) for i in range(5)]
input_imgs = np.copy(orig_imgs)
print(np.shape(input_imgs))

# preprocess input for model
input_imgs = preprocess_input(input_imgs) #for built in keras models

In [ ]:
# Select only one from your list for example
example_id = 2

In [ ]:
plt.imshow(orig_imgs[example_id].astype(int))
plt.xticks([])
plt.yticks([])
plt.tight_layout()
plt.show()

In [ ]:
# Which class you want to target.
target_class = 340 # ImageNet "zebra"

# GradCAM and GuidedGradCAM requires a specific layer
target_layer = "block5_pool" # VGG only

In [ ]:
# Optional
predictions = model.predict(input_imgs)
pred_id = np.argmax(predictions[example_id])
print(decode_predictions(predictions))
print("prediction id:", pred_id)
print("target id:", target_class)

In [ ]:
use_relu = False

# GradCAM

In [ ]:
partial_gradcam_analyzer = GradCAM(
    model=partial_model,
    target_id=target_class,
    layer_name=target_layer,
    relu=use_relu,
)
analysis_partial_grad_cam = partial_gradcam_analyzer.analyze(input_imgs)
heatmap(analysis_partial_grad_cam[example_id].sum(axis=(2)))
plt.show()

# Guided Back Propagation

In [ ]:
guidedbackprop_analyzer = GBP(
    partial_model,
    target_id=target_class,
    relu=use_relu,
)
analysis_guidedbackprop = guidedbackprop_analyzer.analyze(input_imgs)
heatmap(analysis_guidedbackprop[example_id].sum(axis=(2)))
plt.show()

# GuidedGradCAM

In [ ]:
guidedgradcam_analyzer = GuidedGradCAM(
    partial_model,
    target_id=target_class,
    layer_name=target_layer,
    relu=False,
)
analysis_guidedgradcam = guidedgradcam_analyzer.analyze(input_imgs)
heatmap(analysis_guidedgradcam[example_id].sum(axis=(2)))
plt.show()

# LRP

In [ ]:
lrp_analyzer = LRP(
    partial_model,
    target_id=target_class,
    relu=use_relu,
    low=min_input,
    high=max_input,
)
analysis_lrp = lrp_analyzer.analyze(input_imgs)
heatmap(analysis_lrp[example_id].sum(axis=(2)))
plt.show()

# CLRP

In [ ]:
clrp_analyzer = CLRP(
    partial_model,
    target_id=target_class,
    relu=use_relu,
    low=min_input,
    high=max_input,
)
analysis_clrp = clrp_analyzer.analyze(input_imgs)
heatmap(analysis_clrp[example_id].sum(axis=(2)))
plt.show()

# SGLRP

In [ ]:
sglrp_analyzer = SGLRP(
    partial_model,
    target_id=target_class,
    relu=use_relu,
    low=min_input,
    high=max_input,
)
analysis_sglrp = sglrp_analyzer.analyze(input_imgs)
heatmap(analysis_sglrp[example_id].sum(axis=(2)))
plt.show()

# SGLRP Sequential A

In [ ]:
sglrpa_analyzer = SGLRPSeqA(
    partial_model,
    target_id=target_class,
    relu=use_relu,
)
analysis_sglrpa = sglrpa_analyzer.analyze(input_imgs)
heatmap(analysis_sglrpa[example_id].sum(axis=(2)))
plt.show()

# SGLRP Sequential B

In [ ]:
sglrpb_analyzer = SGLRPSeqB(
    partial_model,
    target_id=target_class,
    relu=use_relu,
)
analysis_sglrpb = sglrpb_analyzer.analyze(input_imgs)
heatmap(analysis_sglrpb[example_id].sum(axis=(2)))
plt.show()

# LRP Sequential A

In [ ]:
lrpa_analyzer = LRPA(
    partial_model,
    target_id=target_class,
    relu=use_relu,
)
analysis_lrpa = lrpa_analyzer.analyze(input_imgs)
heatmap(analysis_lrpa[example_id].sum(axis=(2)))
plt.show()

# LRP Sequential B

In [ ]:
lrpb_analyzer = LRPB(
    partial_model,
    target_id=target_class,
    relu=use_relu,
)
analysis_lrpb = lrpb_analyzer.analyze(input_imgs)
heatmap(analysis_lrpb[example_id].sum(axis=(2)))
plt.show()

# LRP Epsilon

In [ ]:
lrpe_analyzer = LRPE(
    partial_model,
    target_id=target_class,
    relu=use_relu,
)
analysis_lrpe = lrpe_analyzer.analyze(input_imgs)
heatmap(analysis_lrpe[example_id].sum(axis=(2)))
plt.show()